# imports

https://www.wrl.unsw.edu.au/news/coastsat-how-to-estimate-beach-slopes-using-satellite-imagery
https://www.wrl.unsw.edu.au/research/coastsat


In [3]:
%load_ext autoreload
%autoreload 2
import os
import numpy as np
import pickle
import warnings
import time
warnings.filterwarnings("ignore")
import matplotlib
matplotlib.use('Qt5Agg')
import matplotlib.pyplot as plt
from matplotlib import gridspec
plt.ion()
import pandas as pd
from datetime import datetime
from coastsat import SDS_earthcache_api, SDS_preprocess, SDS_shoreline, SDS_tools, SDS_transects, SDS_earthcache_client

## setup timeline and AOL

In [4]:
#setup monthly pipelines
def fmt(x):
    if x < 10:
        return '0' + str(x)
    else:
        return str(x)
def monthly_pipes(startyear,endyear=2024):  
    starts = [] ; ends = []
    dates = range(startyear, endyear)  
    for y in dates:
        for m in range(1, 13):
            starts.append(str(y) + '-' + fmt(m) + '-01')
            if m == 2:
                ends.append(str(y) + '-' + fmt(m) + '-28')
            elif m in [4, 6, 9, 11]:
                ends.append(str(y) + '-' + fmt(m) + '-30')
            else:
                ends.append(str(y) + '-' + fmt(m) + '-31')
    return starts, ends

In [5]:
# these coordinates are a rectangle around the island of Madhirivaadhoo
aoi = { "type": "Polygon", 
        "coordinates": [
      [
        [
          72.94835170895487,
          5.464405141242935
        ],
        [
          72.94835170895487,
          5.452180140477154
        ],
        [
          72.96171014465997,
          5.452180140477154
        ],
        [
          72.96171014465997,
          5.464405141242935
        ],
        [
          72.94835170895487,
          5.464405141242935
        ]
      ]
    ]
    }

# setup pipeline

In [6]:
key = input("please enter your API key for Earthcache:")
max_cost=500

In [7]:
repo = 'CoastSat'
root_path = os.getcwd()[ 0 : os.getcwd().find( repo ) + len ( repo )]
cfg_path = os.path.join( root_path, 'earthcache-cfg' )
notebook_client = SDS_earthcache_client.EcClient(cfg_path, key,max_cost)

d5431cc4-b43b-4f31-8db4-d1efe9e589c9


In [8]:

location_name = 'Madhirivaadhoo_'
# need to specify what type of images want to be downloaded
image_type = "a8fc3dde-a3e8-11e7-9793-ae4260ee3b4b" #ID for true color image
# the available IDs can be found at: https://support.skywatch.com/hc/en-us/articles/7297565063067-Available-Outputs
# look at the SDS_earthcache_api.py file for more options
# like searching, creating a pipeline from a search, and getting the max cost of a pipeline

In [9]:
resolution = 'medium'
interval = '1d'

In [10]:
# need to initialize the api first
#SDS_earthcache_api.initialize(key)
#create a set of dates so each pipeline is one month
starts,ends = monthly_pipes(2023)
for i in range(0,1):#len(starts)):
    start_date = starts[i]
    end_date = ends[i]
    pipeline_name = location_name + start_date + '_' + resolution
    print(pipeline_name)
    status, response = notebook_client.calculatePrice(resolution, aoi, start_date, end_date,interval)
    if(status != 200):
        print(response['errors'])
    else:
        print(response['data']['archive'])
        print("balance: "+str(response['data']['prepurchase_balance']))
    #add 3 seconds of delay between each request
    time.sleep(3)

Madhirivaadhoo_2023-01-01_medium
{'interval_quantity': 31, 'price_km2': 2.5, 'minimum_area_km2': 0.1, 'interval_cost': 5.04, 'max_cost': 156.13}
balance: 3000.0


In [11]:
#key = input("please enter your API key for Earthcache:")
SDS_earthcache_api.initialize(key)
for i in range(0,1):
    start_date = starts[i]
    end_date = ends[i]
    pipeline_name = location_name + start_date + '_' + resolution
    SDS_earthcache_api.retrieve_images_earthcache(key, pipeline_name, aoi, start_date, end_date, image_type,interval,resolution)
    time.sleep(3)

d5431cc4-b43b-4f31-8db4-d1efe9e589c9
403 {'errors': [{'message': 'The payment method on file for your account is not valid. Kindly log in and enter a valid credit card on your Account page and try again, or contact customer support at support@skywatch.com.'}]}
200 {'data': {'id': '79e05561-f131-11ee-8136-2dddc51bcd7e', 'org_id': '001JR00000AfBY1YAN', 'name': 'Madhirivaadhoo_2023-01-01_medium', 'start_date': '2023-01-01', 'end_date': '2023-01-31', 'aoi': {'type': 'Polygon', 'coordinates': [[[72.94835170895487, 5.464405141242935], [72.94835170895487, 5.452180140477154], [72.96171014465997, 5.452180140477154], [72.96171014465997, 5.464405141242935], [72.94835170895487, 5.464405141242935]]]}, 'area_km2': 2.0145276052499086, 'cloud_cover_percentage': 100, 'min_aoi_coverage_percentage': 0, 'interval': '1d', 'resolution_low': 30, 'resolution_high': 10, 'output': {'id': 'a8fc3dde-a3e8-11e7-9793-ae4260ee3b4b', 'format': 'geotiff', 'mosaic': 'stitched'}, 'status': 'complete', 'tags': [], 'result

In [27]:
# after about 20 minutes, make sure the status is ready
SDS_earthcache_api.initialize(key)
for i in range(len(starts)):
    start_date = starts[i]
    end_date = ends[i]
    pipeline_name = location_name + start_date + '_' + resolution[0]
    status,res = SDS_earthcache_api.checkStatus(pipeline_name)
    print(status,res)

d5431cc4-b43b-4f31-8db4-d1efe9e589c9
200 {'data': [{'id': '07987725-9768-42c3-815f-a7e38ca6bceb', 'created_at': '2024-04-02T18:19:34.285955+0000', 'updated_at': '2024-04-02T18:26:23.305584+0000', 'pipeline_id': '7efa2c7e-f11d-11ee-85e4-497340b010e0', 'output_id': 'a8fc3dde-a3e8-11e7-9793-ae4260ee3b4b', 'status': 'complete', 'message': 'Processing complete. 1 result available', 'interval': {'start_date': '2023-01-01', 'end_date': '2023-01-01'}, 'total_interval_cost': 0.0, 'overall_metadata': {'scene_height': 132, 'scene_width': 143, 'filled_area_km2': 1.8, 'filled_area_percentage_of_aoi': 90.0, 'cloud_cover_percentage': 2.0, 'cloud_cover_percentage_of_aoi': 2.0, 'visible_area_km2': 1.76, 'visible_area_percentage': 98.0, 'visible_area_percentage_of_aoi': 88.0}, 'results': [{'created_at': '2024-04-02T21:05:16.131831', 'updated_at': '2024-04-02T18:26:23.305584+0000', 'preview_url': 'https://s3-us-west-2.amazonaws.com/aoi-processed-images-prod/7efa2c7e-f11d-11ee-85e4-497340b010e0/87f6234c-8

In [39]:
starts

['2023-01-01',
 '2023-02-01',
 '2023-03-01',
 '2023-04-01',
 '2023-05-01',
 '2023-06-01',
 '2023-07-01',
 '2023-08-01',
 '2023-09-01',
 '2023-10-01',
 '2023-11-01',
 '2023-12-01']

In [41]:
# once the status is good, run this cell to download the images
SDS_earthcache_api.initialize(key)
images = []
for i in range(len(starts)):
    start_date = starts[i]
    pipeline_name = location_name + start_date + '_' + resolution
    status,res = SDS_earthcache_api.checkStatus(pipeline_name)
    if(status == 200):
        images.append(SDS_earthcache_api.download_images(pipeline_name))

d5431cc4-b43b-4f31-8db4-d1efe9e589c9
data/Madhirivaadhoo_2023-01-01_low\07987725-9768-42c3-815f-a7e38ca6bceb
... downloading https://s3-us-west-2.amazonaws.com/aoi-processed-images-prod/7efa2c7e-f11d-11ee-85e4-497340b010e0/87f6234c-8ddb-497a-9087-13ab024cba54/metadata/SKYWATCH_S2_MS_20230101T0522_TC_Tile_0_0_7259_metadata.json
... downloading https://s3-us-west-2.amazonaws.com/aoi-processed-images-prod/7efa2c7e-f11d-11ee-85e4-497340b010e0/87f6234c-8ddb-497a-9087-13ab024cba54/analytic/SKYWATCH_S2_MS_20230101T0522_TC_Tile_0_0_7259.tif
data/Madhirivaadhoo_2023-01-01_low\44266134-c763-4123-8568-681d8c4ae70d
data/Madhirivaadhoo_2023-01-01_low\7344d925-edd8-45ee-86bb-6d2dad79335e
data/Madhirivaadhoo_2023-01-01_low\30386aea-7e66-42d2-95dc-658ae8b693db
data/Madhirivaadhoo_2023-01-01_low\86ccce6b-c55b-4187-a823-bb6d0c28f33c
data/Madhirivaadhoo_2023-01-01_low\9829bd12-2ac9-45b2-81b1-edffc105e6f6
... downloading https://s3-us-west-2.amazonaws.com/aoi-processed-images-prod/7efa2c7e-f11d-11ee-85e4-

In [45]:
images[ 0 ][ 0 ][0]

'data/Madhirivaadhoo_2023-01-01_low\\07987725-9768-42c3-815f-a7e38ca6bceb\\20230101_052229/SKYWATCH_S2_MS_20230101T0522_TC_Tile_0_0_7259.tif'

In [46]:
# this function visualizes the first image
SDS_earthcache_api.view_first_image(images[0])